#### Putting It All Together

As you might have guessed from the last notebook, using all of the variables was allowing you to drastically overfit the training data.  This was great for looking good in terms of your Rsquared on these points.  However, this was not great in terms of how well you were able to predict on the test data.

We will start where we left off in the last notebook.  First read in the dataset.

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import AllTogether as t
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./survey_results_public.csv')
df.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Question 1

**1.** To begin fill in the format function below with the correct variable.  Notice each **{ }** holds a space where one of your variables will be added to the string.  This will give you something to do while the the function does all the steps you did throughout this lesson.

In [4]:
a = 'test_score'
b = 'train_score'
c = 'linear model (lm_model)'
d = 'X_train and y_train'
e = 'X_test'
f = 'y_test'
g = 'train and test data sets'
h = 'overfitting'

q1_piat = '''In order to understand how well our {} fit the dataset, 
            we first needed to split our data into {}.  
            Then we were able to fit our {} on the {}.  
            We could then predict using our {}  by providing 
            the linear model the {} for it to make predictions.  
            These predictions were for {}. 

            By looking at the {}, it looked like we were doing awesome because 
            it was 1!  However, looking at the {} suggested our model was not 
            extending well.  The purpose of this notebook will be to see how 
            well we can get our model to extend to new data.
            
            This problem where our data fits the training data well, but does
            not perform well on test data is commonly known as 
            {}.'''.format(c, g, c, d, c, e, f, b, a, h) #replace a with the correct variable

print(q1_piat)

In order to understand how well our linear model (lm_model) fit the dataset, 
            we first needed to split our data into train and test data sets.  
            Then we were able to fit our linear model (lm_model) on the X_train and y_train.  
            We could then predict using our linear model (lm_model)  by providing 
            the linear model the X_test for it to make predictions.  
            These predictions were for y_test. 

            By looking at the train_score, it looked like we were doing awesome because 
            it was 1!  However, looking at the test_score suggested our model was not 
            extending well.  The purpose of this notebook will be to see how 
            well we can get our model to extend to new data.
            
            This problem where our data fits the training data well, but does
            not perform well on test data is commonly known as 
            overfitting.


In [5]:
# Print the solution order of the letters in the format
t.q1_piat_answer()

This one is tricky - here is the order of the letters for the solution we had in mind:
 c, g, c, d, c, e, f, b, a, h


#### Question 2

**2.** Now, we need to improve the model . Use the dictionary below to provide the true statements about improving **this model**.  **Also consider each statement as a stand alone**.  Though, it might be a good idea after other steps, which would you consider a useful **next step**?

In [8]:
a = 'yes'
b = 'no'

q2_piat = {'add interactions, quadratics, cubics, and other higher order terms': b,#letter here, 
           'fit the model many times with different rows, then average the responses':a, #letter here,
           'subset the features used for fitting the model each time': a,#letter here,
           'this model is hopeless, we should start over': b#letter here
          }

In [9]:
#Check your solution
t.q2_piat_check(q2_piat)

Nice job! That looks right!  These two techniques are really common in Machine Learning algorithms to combat overfitting.  Though the first technique could be useful, it is not likely to help us right away with our current model.  These additional features would likely continue to worsen the nature of overfitting we are seeing here.


##### Question 3

**3.** Before we get too far along, follow the steps in the function below to create the X (explanatory matrix) and y (response vector) to be used in the model.  If your solution is correct, you should see a plot similar to the one shown in the Screencast.

In [33]:
def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''

    
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

def clean_data(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    Perform to obtain the correct X and y objects
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no salaries
    2. Create X as all the columns that are not the Salary column
    3. Create y as the Salary column
    4. Drop the Salary, Respondent, and the ExpectedSalary columns from X
    5. For each numeric variable in X, fill the column with the mean value of the column.
    6. Create dummy columns for all the categorical variables in X, drop the original columns
    '''
    #Drop the rows with missing response values
    df  = df.dropna(subset=['Salary'], axis=0)

    #Drop columns with all NaN values
    df = df.dropna(how='all', axis=1)
    
    X = df.drop('Salary',axis=1)
    y = df['Salary']
    for col in ['Respondent','ExpectedSalary']:
        try:
            X = X.drop(col,axis=1)
        except:
            pass

    cat_df = X.select_dtypes(include=['object'])
    cat_cols_lst = cat_df.columns
    
    not_cat = [x for x in X.columns if not x in cat_cols_lst]
    
    # Mean function
    fill_mean = lambda col: col.fillna(col.mean())
    # Fill the mean
    X_num = X[not_cat].apply(fill_mean, axis=0)
    
    #Dummy categorical variables
    X_cat = create_dummy_df(X, cat_cols_lst, dummy_na=False)
    
    X = pd.concat([X_num, X_cat], axis=1)
    
    return X, y
    
#Use the function to create X and y
X, y = clean_data(df)    

In [34]:
X

,CareerSatisfaction,JobSatisfaction,HoursPerWeek,StackOverflowSatisfaction,CareerSatisfaction,JobSatisfaction,HoursPerWeek,StackOverflowSatisfaction,"ProgramHobby_Yes, I contribute to open source projects","ProgramHobby_Yes, I program as a hobby",...,QuestionsInteresting_Strongly agree,QuestionsInteresting_Strongly disagree,QuestionsConfusing_Disagree,QuestionsConfusing_Somewhat agree,QuestionsConfusing_Strongly agree,QuestionsConfusing_Strongly disagree,InterestedAnswers_Disagree,InterestedAnswers_Somewhat agree,InterestedAnswers_Strongly agree,InterestedAnswers_Strongly disagree
2,8.000000,9.000000,2.447415,8.000000,8.0,9.0,NaN,8.0,0,0,...,0,0,1,0,0,0,0,0,0,0
14,8.000000,8.000000,2.447415,8.000000,8.0,8.0,NaN,8.0,0,1,...,0,0,1,0,0,0,0,0,0,0
17,9.000000,8.000000,2.447415,8.000000,9.0,8.0,NaN,8.0,0,0,...,0,0,1,0,0,0,0,0,0,0
18,5.000000,3.000000,2.447415,8.442686,5.0,3.0,NaN,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0
22,8.000000,9.000000,2.447415,8.000000,8.0,9.0,NaN,8.0,0,0,...,0,0,0,1,0,0,0,0,0,0
25,7.000000,7.000000,0.000000,9.000000,7.0,7.0,0.0,9.0,0,1,...,1,0,1,0,0,0,0,0,1,0
34,10.000000,8.000000,1.000000,10.000000,10.0,8.0,1.0,10.0,0,1,...,0,0,0,0,0,1,0,0,0,0
36,7.000000,9.000000,2.447415,7.000000,7.0,9.0,NaN,7.0,0,1,...,0,0,0,0,0,1,0,0,1,0
37,7.534907,7.024825,1.000000,8.000000,NaN,NaN,1.0,8.0,0,0,...,0,0,1,0,0,0,0,0,0,0
52,6.000000,5.000000,4.000000,8.000000,6.0,5.0,4.0,8.0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [35]:
y

2        113750.000000
14       100000.000000
17       130000.000000
18        82500.000000
22       100764.000000
25       175000.000000
34        14838.709677
36        28200.000000
37       118279.569892
52        15674.203822
54        64516.129032
57        43010.752688
61         6636.323594
64        69892.473118
65        17777.777778
69        32258.064516
70        65000.000000
72       120000.000000
75        46236.559140
76        96283.000000
81       110000.000000
83       165000.000000
89       113694.267516
90        38600.000000
95        53763.440860
97        27548.209366
99       109400.000000
102       83870.967742
103       51000.000000
104       17515.923567
             ...      
18945    121500.000000
18946     38709.677419
18947       405.405405
18950     43750.000000
18955     23750.000000
18964     32962.962963
18984     37500.000000
18985    150000.000000
18989    155000.000000
18990     62500.000000
18996     37500.000000
19004      2040.740741
19005     6

### Run the Cell Below to Acheive the Results Needed for Question 4

In [ ]:
#cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.
cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 25]
    
#Run this cell to pass your X and y to the model for testing
r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test = t.find_optimal_lm_mod(X, y, cutoffs)

#### Question 4

**4.** Use the output and above plot to correctly fill in the keys of the **q4_piat** dictionary with the correct variable.  Notice that only the optimal model results are given back in the above - they are stored in **lm_model**, **X_train**, **X_test**, **y_train**, and **y_test**.  If more than one answer holds, provide a tuple holding all the correct variables in the order of first variable alphabetically to last variable alphabetically.

In [ ]:
# Cell for your computations to answer the next question

In [ ]:
a = 'we would likely have a better rsquared for the test data.'
b = 1000
c = 872
d = 0.69
e = 0.82
f = 0.88
g = 0.72
h = 'we would likely have a better rsquared for the training data.'

q4_piat = {'The optimal number of features based on the results is': #letter here, 
               'The model we should implement in practice has a train rsquared of': #letter here, 
               'The model we should implement in practice has a test rsquared of': #letter here,
               'If we were to allow the number of features to continue to increase': #letter here
}

In [ ]:
#Check against your solution
t.q4_piat_check(q4_piat)

#### Question 5

**5.** The default penalty on coefficients using linear regression in sklearn is a ridge (also known as an L2) penalty.  Because of this penalty, and that all the variables were normalized, we can look at the size of the coefficients in the model as an indication of the impact of each variable on the salary.  The larger the coefficient, the larger the expected impact on salary.  

Use the space below to take a look at the coefficients.  Then use the results to provide the **True** or **False** statements based on the data.

#### Run the below to complete the following dictionary

In [ ]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.head(20)

In [ ]:
a = True
b = False

#According to the data...
q5_piat = {'Country appears to be one of the top indicators for salary': #letter here,
               'Gender appears to be one of the indicators for salary': #letter here, 
               'How long an individual has been programming appears to be one of the top indicators for salary': #letter here,
               'The longer an individual has been programming the more they are likely to earn': #letter here}

In [ ]:
t.q5_piat_check(q5_piat)

#### Congrats of some kind

Congrats!  Hopefully this was a great review, or an eye opening experience about how to put the steps together for an analysis.  List the steps.  In the next lesson, you will look at how take this and show it off to others so they can act on it.